# Colocando o Istio em execução

## Acessando o k8s

Vamos configurar o ambiente, você precisará configurá-lo a cada nova seção ou executar o jupyter passando as variáveis, como descrito no [README](README.md) e não precisará repetir esses comandos novamente.

In [ ]:
# [Opcional] Se você não estiver usando o docker-desktop será necessário obter o arquivo de configuração e ajustar a variável KUBECONFIG
export KUBECONFIG=~/.kube/config # Local do kubeconfig do docker-desktop. Selecione o local onde você colocou o arquivo de config para outras opções (AKS, EKS, GKE, etc).

In [1]:
# [Opcional] Se você tem mais de um cluster na configuração, verifique se está apontando para o correto.
kubectl config get-contexts

CURRENT   NAME             CLUSTER          AUTHINFO         NAMESPACE
*         docker-desktop   docker-desktop   docker-desktop   


In [2]:
# Vamos verificar como estão os nós do nosso cluster:
kubectl get nodes

NAME             STATUS   ROLES    AGE     VERSION
docker-desktop   Ready    master   2m41s   v1.19.7


## Instalando o Istio (linux)

Para instalar a última versão do Istio, neste momento 1.10.0, você pode ir até a página [Getting Started](https://istio.io/latest/docs/setup/getting-started/#download) ou seguir as instruções abaixo:

In [3]:
curl -L https://istio.io/downloadIstio | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   102  100   102    0     0    103      0 --:--:-- --:--:-- --:--:--   103
100  4573  100  4573    0     0   2957      0  0:00:01  0:00:01 --:--:--  2957


Istio 1.10.0 Download Complete!

Istio has been successfully downloaded into the istio-1.10.0 folder on your system.

Next Steps:
See https://istio.io/latest/docs/setup/install/ to add Istio to your Kubernetes cluster.

To configure the istioctl client tool for your workstation,
add the /mnt/d/Projects/istio-curso/istio-1.10.0/bin directory to your environment path variable with:
	 export PATH="$PATH:/mnt/d/Projects/istio-curso/istio-1.10.0/bin"

Begin the Istio pre-installation check by running:
	 istioctl x precheck 

Need more information? Visit https://istio.io/latest/docs/setup/install/ 


Para usar o comando `istioctl`, que está no diretório `bin` do download, coloque-o na variável `PATH` ou copie o arquivo `bin/istioctl` para um diretório no seu `PATH`

Copiando o istioctl para o diretório de binários

O istioctl é um utilitário de linha de comando e foi copiado para a sua máquina durante o download acima.

Para utilizá-lo, basta colocar o arquivo em algum diretório listado na variável PATH.

O istioctl é um utilitário de linha de comando e foi copiado para a sua máquina durante o download acima.

Para utilizá-lo, basta colocar o arquivo em algum diretório listado na variável PATH.

In [4]:
echo $PATH

/home/adsantos/.krew/bin:/mnt/d/miniconda3-wsl/bin:/home/adsantos/.nvm/versions/node/v14.9.0/bin:/home/adsantos/bin:/home/adsantos/anaconda3/envs/py-38/bin:/home/adsantos/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/mnt/c/Program Files (x86)/Common Files/Intel/Shared Libraries/redist/intel64/compiler:/mnt/c/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/bin:/mnt/c/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/libnvvp:/mnt/c/Program Files/ImageMagick-7.0.9-Q16:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0:/mnt/c/Windows/System32/OpenSSH:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Program Files/Git/cmd:/mnt/c/Program Files/dotnet:/mnt/c/ProgramData/chocolatey/bin:/mnt/c/Touch Portal/plugins/adb/platform-tools:/mnt/c/Program Files/NVIDIA Corporation/Nsight Compute 2020.1.1:/mnt/c/Program Files/NVIDIA Corporation/NVIDIA Nv

Os diretórios mais indicados são os do usuário para binários, caso você não tenha, pode criar um e colocá-lo na variável PATH.

A seguir criaremos o diretório `˜/bin`, adicionaremos o caminho na variável PATH e copiaremos o arquivo para o diret

In [8]:
ISTIO_VERSION=1.10.0
# Verificar onde está o executável do Istio
ls istio-$ISTIO_VERSION/bin
export PATH=$PWD/istio-$ISTIO_VERSION/bin:$PATH

istioctl


In [9]:
# Teste
istioctl version

no running Istio pods in "istio-system"
1.10.0


Você também pode copiar o comando em qualquer diretório ou mantê-lo no diretório do Istio e ajustar a variável `PATH` na sessão.

Para tornar essa configuração permanente, você deve adicionar o caminho absoluto (substitua p PWD) a variável PATH no arquivo de configuração do shell do seu usuário, por exemplo ~/.bashrc ou ~/.profile ou ~/.zshrc, etc.

## O mínimo do Istio

Desde a versão 1.6 o Istio é composto de uma única entrega chamada `istiod` e ela pode ser instalada com o comando abaixo:

In [10]:
# Isso deve demorar de 2 a 5 minutos
istioctl install --set profile=minimal --skip-confirmation

Detected that your cluster does not support third party JWT authentication. Falling back to less secure first party JWT. See https://istio.io/v1.10/docs/ops/best-practices/security/#configure-third-party-service-account-tokens for details.
! values.global.jwtPolicy is deprecated; use Values.global.jwtPolicy=third-party-jwt. See http://istio.io/latest/docs/ops/best-practices/security/#configure-third-party-service-account-tokens for more information instead
✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Installation complete                                                         Thank you for installing Istio 1.10.  Please take a few minutes to tell us about your install/upgrade experience!  https://forms.gle/KjkrDnMPByq7akrYA

> Em Mar-2021 não é possível instalar o Istio no GKE Autopilot, ele retorna o erro: `Istiod encountered an error: failed to update resource with server-side apply for obj MutatingWebhookConfiguration`. Veja mais sobre [incopatibilidades do GKE Autopilot](https://cloud.google.com/kubernetes-engine/docs/concepts/autopilot-overview?hl=pt-br#unsupported_cluster_features).
> Para GKE crie um cluster padrão Zonal.

Vamos verificar o que foi instalado

In [11]:
# Obtendo os namespaces
kubectl get ns

# Verificando o namespace do Istio
kubectl get all -n istio-system

NAME              STATUS   AGE
default           Active   4m42s
istio-system      Active   18s
kube-node-lease   Active   4m44s
kube-public       Active   4m44s
kube-system       Active   4m44s
NAME                          READY   STATUS    RESTARTS   AGE
pod/istiod-58fd54c6d5-gbj89   1/1     Running   0          17s

NAME             TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                                 AGE
service/istiod   ClusterIP   10.101.70.250   <none>        15010/TCP,15012/TCP,443/TCP,15014/TCP   17s

NAME                     READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/istiod   1/1     1            1           18s

NAME                                DESIRED   CURRENT   READY   AGE
replicaset.apps/istiod-58fd54c6d5   1         1         1       18s

NAME                                         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
horizontalpodautoscaler.autoscaling/istiod   Deployment/istiod   <unknown>/80%   1         5  

Instalamos apenas o plano de controle e mais adiante instalaremos o ingress, egress e alguns adicionais como grafana, kiali e jaeger, mas já temos o Istio instalado e pronto para gerenciar os Envoys.

## Istio CRDs

O Istio é uma aplicação que é executada no kubernetes e adiciona recursos personalizados ([CRD](https://kubernetes.io/docs/concepts/extend-kubernetes/api-extension/custom-resources/)) para sua configuração.

Você pode obter uma lista completa executando o comando abaixo:

In [12]:
kubectl api-resources --api-group=networking.istio.io

kubectl api-resources --api-group=security.istio.io

NAME               SHORTNAMES   APIGROUP              NAMESPACED   KIND
destinationrules   dr           networking.istio.io   true         DestinationRule
envoyfilters                    networking.istio.io   true         EnvoyFilter
gateways           gw           networking.istio.io   true         Gateway
serviceentries     se           networking.istio.io   true         ServiceEntry
sidecars                        networking.istio.io   true         Sidecar
virtualservices    vs           networking.istio.io   true         VirtualService
workloadentries    we           networking.istio.io   true         WorkloadEntry
workloadgroups     wg           networking.istio.io   true         WorkloadGroup
NAME                     SHORTNAMES   APIGROUP            NAMESPACED   KIND
authorizationpolicies                 security.istio.io   true         AuthorizationPolicy
peerauthentications      pa           security.istio.io   true         PeerAuthentication
requestauthentications   ra        

Esses são os recursos adicionados e na segunda coluna o nome abreviado, você pode utilizar um ou outro, por exemplo:

In [13]:
# O comando
kubectl get svc

# É equivalente ao

kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4m50s
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4m50s


> Note que todos as configurações para recursos do Istio são aplicados para um _namespace_ (Namespaced=true), isso significa que a configuração terá efeito apenas em um _namespace_, porém, configurações que poderão ser aplicadas para toda a malha de serviços serão realizadas no _namespace_ do Istio, o `istio-system`.

Você verá muito disso no kubernetes, para uma lista completa de recursos e suas abreviações execute o mesmo comando acima, mas sem o filtro `kubectl api-resources`

In [14]:
kubectl api-resources

NAME                              SHORTNAMES   APIGROUP                       NAMESPACED   KIND
bindings                                                                      true         Binding
componentstatuses                 cs                                          false        ComponentStatus
configmaps                        cm                                          true         ConfigMap
endpoints                         ep                                          true         Endpoints
events                            ev                                          true         Event
limitranges                       limits                                      true         LimitRange
namespaces                        ns                                          false        Namespace
nodes                             no                                          false        Node
persistentvolumeclaims            pvc                                         true         Persisten

> **Dica pro**: Quem utiliza o kubectl com muita frequência costuma criar apelidos para os comandos mais comuns, por exemplo:

In [15]:
# Alias para o comando kubectl
alias k=kubectl
# Teste
k get svc
# Alias para o comando kubectl get
alias kg='kubectl get'
# Teste
kg svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4m59s
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4m59s


> Você pode colocar esses comandos nos arquivos de configuração do shell (.profile, .bashrc, etc), para torná-los persistentes entre sessões.

## Ativando o Istio para um namespace

Um elemento chave do Istio é o _sidecar_ ou _proxy_, como veremos na discussão sobre arquitetura do Istio. Para que o Istio injete-o nos PODs da aplicação, é necessário marcar o _namespace_ da seguinte forma:

In [16]:
kubectl label namespace default istio-injection=enabled

namespace/default labeled


Vamos verificar como ficou a configuração do namespace

In [17]:
kubectl describe ns/default

Name:         default
Labels:       istio-injection=enabled
Annotations:  <none>
Status:       Active

No resource quota.

No LimitRange resource.


O `istiod` irá monitorar todos os _namespaces_ com o rótulo `istio-injection=enabled` e adicionar ao POD um _container_ de proxy, o _sidecar_.

Vamos fazer _deploy_ de uma aplicação exemplo para verificar esse comportamento.

## Criando uma aplicação demo

Vamos criar uma aplicação simples.

In [5]:
# Vamos criar um diretório para nossos exemplos:
mkdir -p assets/exemplos/simple-app

# O arquivo de Deployment:
cat <<EOT > assets/exemplos/simple-app/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: v1
spec:
  replicas: 1
  selector:
    matchLabels:
        app: simple-app
        version: v1
  template:
    metadata:
      labels:
        app: simple-app
        version: v1
    spec:
      containers:
        - name: simple-app
          image: "nginx:stable"
          imagePullPolicy: IfNotPresent
          ports:
            - name: http
              containerPort: 80
              protocol: TCP
EOT

# O de serviço
cat <<EOT > assets/exemplos/simple-app/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: v1
spec:
  type: ClusterIP
  ports:
    - port: 80
      targetPort: http
      protocol: TCP
      name: http
  selector:
    app: simple-app
    version: v1
EOT

In [6]:
# Listando os arquivos criados:
ls -la assets/exemplos/simple-app

total 16
drwxr-xr-x  4 anderson  staff  128 Apr 10 13:14 .
drwxr-xr-x  6 anderson  staff  192 Apr 10 13:14 ..
-rw-r--r--  1 anderson  staff  518 Apr 14 16:29 deployment.yaml
-rw-r--r--  1 anderson  staff  260 Apr 14 16:29 service.yaml


Criamos dois arquivos, o `deployment.yaml` e o `service.yaml` no diretório `assets/exemplos/simple-app`, agora vamos instala-la no _namespace_ default (quando omitido é onde os recursos serão criados).

Inspecione os arquivos e tente descobrir o que será instalado no cluster, uma dica, procure a pela imagem.

In [8]:
kubectl apply -f assets/exemplos/simple-app

deployment.apps/simple-app created
service/simple-app created


Verificando a situação do POD.

In [10]:
kubectl get pods

NAME                         READY   STATUS    RESTARTS   AGE
simple-app-cf5fc5558-xvggn   2/2     Running   0          49s


> Para instalar a aplicação em um _namespace_ diferente adicione `--namesapce` ou `-n`ao comando. Exemplo: `kubectl apply -f exemplos/simple-app -n test-app`

Vamos acessar nossa aplicação, ela foi configurada para o tipo de serviço `ClusterIP`, o que significa que o acesso é interno, apenas entre os PODs do cluster, mas podemos acessá-la utilizando o comando `kubectl port-forward`.

In [11]:
kubectl port-forward svc/simple-app 8000:80

Forwarding from 127.0.0.1:8000 -> 80
Forwarding from [::1]:8000 -> 80
Handling connection for 8000
Handling connection for 8000


Vá até o seu navegador e entre acesse a url: <http://localhost:8000>

Pronto, você tem acesso à sua aplicação como se estivesse sendo executada na sua máquina. Claro que o kubernetes pode estar na sua máquina, mas isso funcionará em qualquer kubernetes, local ou remoto.

Para interromper:
* No Jupyter Lab ou Notebook: Clique no icone <kbd>◾</kbd> (_Interrup the kernel_ na barra de ferramentas)
* No terminal: tecle <kbd>CTRL</kbd>+<kbd>C</kbd>

Verificando o que foi instalado.

In [12]:
kubectl get --show-labels all

NAME                             READY   STATUS    RESTARTS   AGE   LABELS
pod/simple-app-cf5fc5558-xvggn   2/2     Running   0          68s   app=simple-app,istio.io/rev=default,pod-template-hash=cf5fc5558,security.istio.io/tlsMode=istio,service.istio.io/canonical-name=simple-app,service.istio.io/canonical-revision=v1,version=v1

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)   AGE   LABELS
service/kubernetes   ClusterIP   10.8.144.1     <none>        443/TCP   41d   component=apiserver,provider=kubernetes
service/simple-app   ClusterIP   10.8.146.141   <none>        80/TCP    67s   app=simple-app,version=v1

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE   LABELS
deployment.apps/simple-app   1/1     1            1           69s   app=simple-app,version=v1

NAME                                   DESIRED   CURRENT   READY   AGE   LABELS
replicaset.apps/simple-app-cf5fc5558   1         1         1       69s   app=simple-app,pod-template-hash=cf5

Note que o pod/simple-app tem 2/2 na coluna pronto (Ready), isso significa que dois containers de dois estão ok, vamos verificar quem é o segundo container.

In [13]:
# Usaremos um dos labels do pod para encontra-lo
kubectl describe pod -l app=simple-app

Name:         simple-app-cf5fc5558-xvggn
Namespace:    default
Priority:     0
Node:         gke-cluster-1-default-pool-1fb77211-sdbr/10.128.0.18
Start Time:   Wed, 14 Apr 2021 16:29:38 -0300
Labels:       app=simple-app
              istio.io/rev=default
              pod-template-hash=cf5fc5558
              security.istio.io/tlsMode=istio
              service.istio.io/canonical-name=simple-app
              service.istio.io/canonical-revision=v1
              version=v1
Annotations:  kubectl.kubernetes.io/default-logs-container: simple-app
              prometheus.io/path: /stats/prometheus
              prometheus.io/port: 15020
              prometheus.io/scrape: true
              sidecar.istio.io/status:
                {"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-...
Status:       Running
IP:           10.4.1.5
IPs:
  IP:           10.4.1.5
Controlled By:  ReplicaSet/simple-app-cf5fc55

É uma grande quantidade de informação, vamos procurar uma seção chamada `Containers` e nela a nossa aplicação `simple-app`.

Como você pode ver, a imagem desse container é `nginx`, com a tag `stable`, mais abaixo tem um segundo container `istio-proxy`, com a imagem `docker.io/istio/proxyv2` e a _tag_ para a  versão `1.10.0`.

Esse container não faz parte do [assets/exemplos/simple-app/deployment.yaml](exemplos/2_simple-app/deployment.yaml), ele foi adicionado ao seu pod pelo `istiod`.

Caso você precise saber todos os _namespaces_ que tem a injeção do _conteiner_ _proxy_ do Istio ativado, basta executar o comando:

In [14]:
kubectl get ns -l istio-injection=enabled

kubectl get pods

NAME      STATUS   AGE
default   Active   41d
NAME                         READY   STATUS    RESTARTS   AGE
simple-app-cf5fc5558-xvggn   2/2     Running   0          2m34s


Até o momento, somente o namespace `default`, onde configuramos a marcação, foi encontrado.

## Conclusão

Neste ponto temos o mínimo do Istio em execução no nosso cluster, mas com exceção de um container extra isso não significa muito.

Na próxima parte iremos explorar os recursos que esta instalação mínima do Istio pode oferecer.